# Support Vector Machines (SVM) i XGBoost 



## Wprowadzenie

W tym notebooku poznamy dwa potężne algorytmy uczenia maszynowego:

### Support Vector Machines (SVM)
![SVM Concept](https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_margin_001.png)

**SVM** to algorytm znajdowania optymalnej granicy decyzyjnej, który maksymalizuje margines między klasami.

### XGBoost
![XGBoost Concept](https://aiml.com/wp-content/uploads/2024/06/evolution-of-boosting.png)

**XGBoost** to zaawansowana implementacja gradient boosting, która buduje model z wielu słabych uczniów.

---

Oba algorytmy mają różne podejścia do rozwiązywania problemów klasyfikacji i regresji, co czyni je komplementarnymi narzędziami w arsenale data scientist'a.

In [ ]:
# Import niezbędnych bibliotek
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_circles, make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

# Ustawienia wykresów
plt.style.use('default')
sns.set_palette("husl")

# Support Vector Machines (SVM)

## Teoria i podstawy

![SVM Visualization](https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/SVM_margin.png/512px-SVM_margin.png)

**Support Vector Machine (SVM)** to algorytm nadzorowanego uczenia maszynowego, który:
- Szuka **hiperplanu** (granicy decyzyjnej) najlepiej oddzielającego klasy
- **Maksymalizuje margines** - odległość od najbliższych punktów każdej klasy
- Może tworzyć **nieliniowe granice** dzięki funkcjom jądra (kernelom)

### Kluczowe koncepcje:
- **Wektory nośne (Support Vectors)** - punkty najbliższe granicy decyzyjnej
- **Margines** - "šszerokość" strefy separującej klasy
- **Parametr C** - kontroluje trade-off między maksymalizacją marginesu a minimalizacją błędów
- **Kernele** - umożliwiają modelowanie nieliniowych zależności

### Typy kerneli:
![Kernel Types](https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_kernels_001.png)

1. **Linear Kernel** - dla danych liniowo separowalnych
2. **RBF (Radial Basis Function)** - najczęściej używany dla danych nieliniowych
3. **Polynomial Kernel** - dla złożonych zależności wielomianowych

In [ ]:
# Funkcja do wizualizacji granic decyzyjnych
def plot_decision_boundary(model, X, y, title="", ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    # Tworzenie siatki punktów
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
    
    # Predykcja dla każdego punktu siatki
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Rysowanie granicy decyzyjnej
    ax.contourf(xx, yy, Z, alpha=0.4, cmap=plt.cm.RdYlBu)
    
    # Rysowanie punktów danych
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    
    return ax

## Przykład 1: SVM na danych liniowo separowalnych

Zaczniemy od prostego przykładu z danymi, które można rozdzielić linią prostą.

In [ ]:
# Generowanie danych liniowo separowalnych
np.random.seed(42)
X_linear, y_linear = make_classification(n_samples=100, n_features=2, n_redundant=0, 
                                       n_informative=2, n_clusters_per_class=1, 
                                       random_state=42)

# Podział na zbiory treningowy i testowy
X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(
    X_linear, y_linear, test_size=0.3, random_state=42)

# Standardyzacja danych
scaler = StandardScaler()
X_train_lin_scaled = scaler.fit_transform(X_train_lin)
X_test_lin_scaled = scaler.transform(X_test_lin)

print(f"Rozmiar zbioru treningowego: {X_train_lin_scaled.shape}")
print(f"Rozmiar zbioru testowego: {X_test_lin_scaled.shape}")

In [ ]:
# Trenowanie SVM z kernelem liniowym
svm_linear = SVC(kernel='linear', C=1.0, random_state=42)
svm_linear.fit(X_train_lin_scaled, y_train_lin)

# Predykcja i ocena
y_pred_lin = svm_linear.predict(X_test_lin_scaled)
accuracy_lin = accuracy_score(y_test_lin, y_pred_lin)

print(f"Dokładność SVM liniowego: {accuracy_lin:.3f}")

# Wizualizacja
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Dane oryginalne
ax1.scatter(X_linear[:, 0], X_linear[:, 1], c=y_linear, cmap=plt.cm.RdYlBu, edgecolors='black')
ax1.set_title('Dane oryginalne')
ax1.set_xlabel('Feature 1')
ax1.set_ylabel('Feature 2')

# Granica decyzyjna
plot_decision_boundary(svm_linear, X_train_lin_scaled, y_train_lin, 
                      'SVM Linear - Granica decyzyjna', ax2)

plt.tight_layout()
plt.show()

## Przykład 2: SVM na danych nieliniowo separowalnych

Teraz sprawdzimy, jak SVM radzi sobie z danymi, które nie mogą być rozdzielone linią prostą. Użyjemy danych w kształcie okręgów.

In [ ]:
# Generowanie danych w kształcie okręgów (nieliniowo separowalne)
X_circles, y_circles = make_circles(n_samples=300, factor=0.5, noise=0.1, random_state=42)

# Podział na zbiory
X_train_circ, X_test_circ, y_train_circ, y_test_circ = train_test_split(
    X_circles, y_circles, test_size=0.3, random_state=42)

print(f"Rozmiar zbioru treningowego: {X_train_circ.shape}")
print(f"Rozmiar zbioru testowego: {X_test_circ.shape}")

# Wizualizacja danych
plt.figure(figsize=(8, 6))
plt.scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap=plt.cm.RdYlBu, edgecolors='black')
plt.title('Dane w kształcie okręgów')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

In [ ]:
# Porównanie różnych kerneli SVM
kernels = ['linear', 'rbf', 'poly']
C_value = 1.0

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, kernel in enumerate(kernels):
    # Trenowanie modelu
    if kernel == 'poly':
        svm = SVC(kernel=kernel, C=C_value, degree=3, random_state=42)
    else:
        svm = SVC(kernel=kernel, C=C_value, random_state=42)
    
    svm.fit(X_train_circ, y_train_circ)
    
    # Predykcja i ocena
    y_pred = svm.predict(X_test_circ)
    accuracy = accuracy_score(y_test_circ, y_pred)
    
    # Wizualizacja
    plot_decision_boundary(svm, X_circles, y_circles, 
                          f'SVM {kernel.capitalize()} (Acc: {accuracy:.3f})', axes[i])
    
    print(f"Dokładność SVM {kernel}: {accuracy:.3f}")

plt.tight_layout()
plt.show()

## Wpływ hiperparametrów SVM

![SVM Parameters](https://scikit-learn.org/stable/_images/sphx_glr_plot_rbf_parameters_001.png)

### Parametr C (Regularyzacja)
- **Wysokie C**: Model stara się klasyfikować wszystkie punkty poprawnie → może prowadzić do overfitting
- **Niskie C**: Model akceptuje więcej błędów, ale ma szerszy margines → może prowadzić do underfitting



### Parametr gamma (dla kernela RBF)
- **Wysokie gamma**: Wpływ pojedynczego punktu jest bardzo lokalny → skomplikowane granice
- **Niskie gamma**: Wpływ punktów jest szerszy → gładsze granice

![Gamma Effect](https://miro.medium.com/v2/resize:fit:1400/1*vmLmExiS9n5pY4Xy9Z516g.png)

### Zasady doboru parametrów:
1. **Małe zbiory danych** → niższe C, wyższe gamma
2. **Duże zbiory danych** → wyższe C, niższe gamma
3. **Szumy w danych** → niższe C

In [ ]:
# Badanie wpływu parametru C
C_values = [0.1, 1, 10, 100]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, C in enumerate(C_values):
    svm = SVC(kernel='rbf', C=C, random_state=42)
    svm.fit(X_train_circ, y_train_circ)
    
    y_pred = svm.predict(X_test_circ)
    accuracy = accuracy_score(y_test_circ, y_pred)
    
    plot_decision_boundary(svm, X_circles, y_circles, 
                          f'SVM RBF (C={C}, Acc: {accuracy:.3f})', axes[i])
    
    print(f"C={C}: Dokładność = {accuracy:.3f}")

plt.tight_layout()
plt.show()

In [ ]:
# Badanie wpływu parametru gamma
gamma_values = [0.1, 1, 10, 100]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, gamma in enumerate(gamma_values):
    svm = SVC(kernel='rbf', C=1, gamma=gamma, random_state=42)
    svm.fit(X_train_circ, y_train_circ)
    
    y_pred = svm.predict(X_test_circ)
    accuracy = accuracy_score(y_test_circ, y_pred)
    
    plot_decision_boundary(svm, X_circles, y_circles, 
                          f'SVM RBF (γ={gamma}, Acc: {accuracy:.3f})', axes[i])
    
    print(f"gamma={gamma}: Dokładność = {accuracy:.3f}")

plt.tight_layout()
plt.show()

## Optymalizacja hiperparametrów SVM

Użyjemy Grid Search do znalezienia optymalnych parametrów.

In [ ]:
# Grid Search dla SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['rbf']
}

svm_grid = SVC(random_state=42)
grid_search = GridSearchCV(svm_grid, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_circ, y_train_circ)

print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepszy wynik CV:", grid_search.best_score_)

# Ocena na zbiorze testowym
best_svm = grid_search.best_estimator_
y_pred_best = best_svm.predict(X_test_circ)
test_accuracy = accuracy_score(y_test_circ, y_pred_best)
print(f"Dokładność na zbiorze testowym: {test_accuracy:.3f}")

In [ ]:
# Wizualizacja wyników Grid Search
plt.figure(figsize=(10, 8))

# Tworzenie macierzy wyników
C_vals = param_grid['C']
gamma_vals = param_grid['gamma']
scores = np.zeros((len(gamma_vals), len(C_vals)))

for i, gamma in enumerate(gamma_vals):
    for j, C in enumerate(C_vals):
        # Znajdź wynik dla tej kombinacji
        for result in grid_search.cv_results_['params']:
            if result['C'] == C and result['gamma'] == gamma:
                idx = grid_search.cv_results_['params'].index(result)
                scores[i, j] = grid_search.cv_results_['mean_test_score'][idx]
                break

# Heatmapa
sns.heatmap(scores, annot=True, fmt='.3f', 
            xticklabels=[f'C={c}' for c in C_vals],
            yticklabels=[f'γ={g}' for g in gamma_vals],
            cmap='viridis')
plt.title('Grid Search Results - Średnia dokładność CV')
plt.xlabel('Parametr C')
plt.ylabel('Parametr gamma')
plt.show()

![Boosting Process](https://cdn.corporatefinanceinstitute.com/assets/boosting1.png)




### XGBoost - ulepszenia:


- **Wydajność**: Optymalizacje algorytmiczne i możliwość przetwarzania równoległego
- **Regularyzacja**: Wbudowane mechanizmy przeciwdziałające overfitting
- **Elastyczność**: Obsługa różnych typów problemów i metryk
- **Braki danych**: Automatyczne radzenie sobie z missing values

### Algorytm krok po kroku:
1. **Inicjalizacja**: Zacznij z prostym modelem (np. średnia)
2. **Oblicz residua**: Znajdź błędy obecnego modelu
3. **Trenuj nowe drzewo**: Na residuach poprzedniego kroku
4. **Dodaj do ensemble**: Z odpowiednią wagą (learning_rate)
5. **Powtórz**: Aż do osiągnięcia założonej liczby drzew

In [ ]:
# Przykład z danymi "moons" dla XGBoost
X_moons, y_moons = make_moons(n_samples=300, noise=0.2, random_state=42)

# Podział na zbiory
X_train_moons, X_test_moons, y_train_moons, y_test_moons = train_test_split(
    X_moons, y_moons, test_size=0.3, random_state=42)

# Wizualizacja danych
plt.figure(figsize=(8, 6))
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap=plt.cm.RdYlBu, edgecolors='black')
plt.title('Dane "moons"')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

print(f"Rozmiar zbioru treningowego: {X_train_moons.shape}")
print(f"Rozmiar zbioru testowego: {X_test_moons.shape}")

In [ ]:
# Trenowanie XGBoost z domyślnymi parametrami
xgb_default = XGBClassifier(random_state=42, eval_metric='logloss')
xgb_default.fit(X_train_moons, y_train_moons)

# Predykcja i ocena
y_pred_xgb = xgb_default.predict(X_test_moons)
accuracy_xgb = accuracy_score(y_test_moons, y_pred_xgb)

print(f"Dokładność XGBoost (domyślne): {accuracy_xgb:.3f}")

# Wizualizacja
plt.figure(figsize=(8, 6))
plot_decision_boundary(xgb_default, X_moons, y_moons, 
                      f'XGBoost - Granica decyzyjna (Acc: {accuracy_xgb:.3f})')
plt.show()

## Kluczowe hiperparametry XGBoost


### Parametry sterujące złożonością modelu:
- **n_estimators**: Liczba drzew w ensemble
- **max_depth**: Maksymalna głębokość pojedynczego drzewa
- **learning_rate (eta)**: Tempo uczenia - skaluje wkład każdego drzewa

### Wpływ max_depth:

- **Głębokie drzewa (depth > 6)**: Mogą prowadzić do overfitting
- **Płytkie drzewa (depth < 3)**: Mogą prowadzić do underfitting
- **Optymalne (depth 3-6)**: Dobry balans bias-variance

### Parametry regularyzacji:
- **subsample**: Frakcja próbek używana do trenowania każdego drzewa
- **colsample_bytree**: Frakcja cech używana do trenowania każdego drzewa
- **reg_alpha**: Regularyzacja L1
- **reg_lambda**: Regularyzacja L2



In [ ]:
# Badanie wpływu max_depth
max_depths = [1, 3, 6, 10]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, max_depth in enumerate(max_depths):
    xgb_model = XGBClassifier(max_depth=max_depth, n_estimators=100, 
                             learning_rate=0.1, random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train_moons, y_train_moons)
    
    # Ocena na zbiorach treningowym i testowym
    train_acc = xgb_model.score(X_train_moons, y_train_moons)
    test_acc = xgb_model.score(X_test_moons, y_test_moons)
    
    plot_decision_boundary(xgb_model, X_moons, y_moons, 
                          f'XGBoost (depth={max_depth})\nTrain: {train_acc:.3f}, Test: {test_acc:.3f}', 
                          axes[i])
    
    print(f"max_depth={max_depth}: Train={train_acc:.3f}, Test={test_acc:.3f}")

plt.tight_layout()
plt.show()

In [ ]:
# Badanie wpływu learning_rate
learning_rates = [0.01, 0.1, 0.3, 1.0]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for i, lr in enumerate(learning_rates):
    xgb_model = XGBClassifier(learning_rate=lr, n_estimators=100, 
                             max_depth=3, random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train_moons, y_train_moons)
    
    train_acc = xgb_model.score(X_train_moons, y_train_moons)
    test_acc = xgb_model.score(X_test_moons, y_test_moons)
    
    plot_decision_boundary(xgb_model, X_moons, y_moons, 
                          f'XGBoost (lr={lr})\nTrain: {train_acc:.3f}, Test: {test_acc:.3f}', 
                          axes[i])
    
    print(f"learning_rate={lr}: Train={train_acc:.3f}, Test={test_acc:.3f}")

plt.tight_layout()
plt.show()

## Optymalizacja hiperparametrów XGBoost

In [ ]:
# Grid Search dla XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgb_grid = XGBClassifier(random_state=42, eval_metric='logloss')
grid_search_xgb = GridSearchCV(xgb_grid, param_grid_xgb, cv=5, 
                              scoring='accuracy', n_jobs=-1, verbose=1)

print("Rozpoczynam Grid Search dla XGBoost...")
grid_search_xgb.fit(X_train_moons, y_train_moons)

print("Najlepsze parametry XGBoost:", grid_search_xgb.best_params_)
print("Najlepszy wynik CV:", grid_search_xgb.best_score_)

# Ocena na zbiorze testowym
best_xgb = grid_search_xgb.best_estimator_
y_pred_best_xgb = best_xgb.predict(X_test_moons)
test_accuracy_xgb = accuracy_score(y_test_moons, y_pred_best_xgb)
print(f"Dokładność na zbiorze testowym: {test_accuracy_xgb:.3f}")

In [ ]:
# Porównanie wyników przed i po optymalizacji
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Model domyślny
plot_decision_boundary(xgb_default, X_moons, y_moons, 
                      f'XGBoost - Domyślne parametry\n(Acc: {accuracy_xgb:.3f})', ax1)

# Model zoptymalizowany
plot_decision_boundary(best_xgb, X_moons, y_moons, 
                      f'XGBoost - Zoptymalizowane parametry\n(Acc: {test_accuracy_xgb:.3f})', ax2)

plt.tight_layout()
plt.show()

## Analiza ważności cech w XGBoost

XGBoost automatycznie oblicza ważność cech, co pomaga w interpretacji modelu.

In [ ]:
# Przykład z większą liczbą cech
X_complex, y_complex = make_classification(n_samples=1000, n_features=10, 
                                         n_informative=5, n_redundant=2,
                                         n_clusters_per_class=1, random_state=42)

feature_names = [f'Feature_{i+1}' for i in range(X_complex.shape[1])]

# Podział danych
X_train_comp, X_test_comp, y_train_comp, y_test_comp = train_test_split(
    X_complex, y_complex, test_size=0.3, random_state=42)

# Trenowanie modelu
xgb_complex = XGBClassifier(n_estimators=100, max_depth=4, 
                           learning_rate=0.1, random_state=42, eval_metric='logloss')
xgb_complex.fit(X_train_comp, y_train_comp)

# Ważność cech
feature_importance = xgb_complex.feature_importances_

# Wizualizacja ważności cech
plt.figure(figsize=(10, 6))
indices = np.argsort(feature_importance)[::-1]
plt.bar(range(len(feature_importance)), feature_importance[indices])
plt.xlabel('Cechy')
plt.ylabel('Ważność')
plt.title('Ważność cech w XGBoost')
plt.xticks(range(len(feature_importance)), [feature_names[i] for i in indices], rotation=45)
plt.tight_layout()
plt.show()

# Wydrukowanie ważności
print("Ważność cech (posortowane):")
for i in indices:
    print(f"{feature_names[i]}: {feature_importance[i]:.4f}")

# Porównanie SVM vs XGBoost


## Kiedy używać SVM:


- **Dane o wysokiej wymiarowości** (np. text mining, image recognition)
- **Małe zbiory danych** (SVM działa dobrze z mniejszą ilością danych)
- **Potrzeba teoretycznego uzasadnienia** (silne fundamenty matematyczne)
- **Problemy z wyraźną separacją klas**
- **Stabilność wyników** (deterministyczny)

## Kiedy używać XGBoost:


- **Duże zbiory danych** (skaluje się lepiej)
- **Dane tabelaryczne** (strukturalne, mixed features)
- **Potrzeba interpretacji** (feature importance)
- **Konkursy ML** (często wygrywa kaggle competitions)
- **Automatic feature engineering** (może znajdować interakcje)
- **Obsługa braków danych** (native missing value handling)

## Zestawienie właściwości:

| Cecha | SVM | XGBoost |
|-------|-----|----------|
| **Typ modelu** | Kernel-based | Tree ensemble |
| **Interpretowalność** | Niska | Wysoka (feature importance) |
| **Szybkość trenowania** | Wolna na dużych danych | Szybka |
| **Odporność na overfitting** | Wysoka (z regularyzacją) | Średnia (wymaga tuningu) |
| **Obsługa braków danych** | Nie | Tak |
| **Hiperparametry** | Mało, ale kluczowe | Dużo opcji do tuningu |
| **Pamięć** | Efektywna | Może być intensywna |

In [ ]:
# Porównanie czasów trenowania i dokładności
import time

# Przygotowanie danych różnej wielkości
sizes = [100, 500, 1000, 2000]
svm_times = []
xgb_times = []
svm_accuracies = []
xgb_accuracies = []

for size in sizes:
    print(f"\nTestowanie na {size} próbkach...")
    
    # Generowanie danych
    X_comp, y_comp = make_classification(n_samples=size, n_features=10, 
                                       n_informative=7, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X_comp, y_comp, 
                                                       test_size=0.3, random_state=42)
    
    # Standardyzacja dla SVM
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Test SVM
    start_time = time.time()
    svm = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
    svm.fit(X_train_scaled, y_train)
    svm_time = time.time() - start_time
    svm_acc = svm.score(X_test_scaled, y_test)
    
    # Test XGBoost
    start_time = time.time()
    xgb_model = XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    xgb_time = time.time() - start_time
    xgb_acc = xgb_model.score(X_test, y_test)
    
    svm_times.append(svm_time)
    xgb_times.append(xgb_time)
    svm_accuracies.append(svm_acc)
    xgb_accuracies.append(xgb_acc)
    
    print(f"SVM: {svm_time:.3f}s, Acc: {svm_acc:.3f}")
    print(f"XGB: {xgb_time:.3f}s, Acc: {xgb_acc:.3f}")

# Wizualizacja porównania
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Czasy trenowania
ax1.plot(sizes, svm_times, 'o-', label='SVM', linewidth=2, markersize=8)
ax1.plot(sizes, xgb_times, 's-', label='XGBoost', linewidth=2, markersize=8)
ax1.set_xlabel('Rozmiar zbioru danych')
ax1.set_ylabel('Czas trenowania (s)')
ax1.set_title('Porównanie czasów trenowania')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Dokładności
ax2.plot(sizes, svm_accuracies, 'o-', label='SVM', linewidth=2, markersize=8)
ax2.plot(sizes, xgb_accuracies, 's-', label='XGBoost', linewidth=2, markersize=8)
ax2.set_xlabel('Rozmiar zbioru danych')
ax2.set_ylabel('Dokładność')
ax2.set_title('Porównanie dokładności')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Zadania praktyczne


## Zadanie 1: SVM na różnych typach danych 🎯


1. **Wygeneruj 3 różne typy syntetycznych danych**:
   - Liniowo separowalne (`make_classification`)
   - Okręgi (`make_circles`)
   - Półksiężyce (`make_moons`)

2. **Dla każdego typu danych**:
   - Przetestuj różne kernele SVM (linear, rbf, poly)
   - Znajdź optymalne parametry używając GridSearchCV
   - Porównaj wyniki i zastanowi się, dlaczego niektóre kernele działają lepiej

**Wskazówka**: Użyj funkcji `plot_decision_boundary` z notebooka!

---

## Zadanie 2: XGBoost - feature engineering 🔧


1. **Załaduj prawdziwy zbiór danych** (np. z sklearn.datasets)
2. **Stwórz nowe cechy przez**:
   - Kombinacje liniowe istniejących cech (`feature1 + feature2`)
   - Interakcje między cechami (`feature1 * feature2`)
   - Transformacje nieliniowe (`log`, `sqrt`, `square`, itp.)
3. **Porównaj wyniki** przed i po feature engineering
4. **Przeanalizuj feature importance** - które nowe cechy są najważniejsze?

---

## Zadanie 3: Porównanie algorytmów ⚔️

![Algorithm Comparison](https://scikit-learn.org/stable/_images/sphx_glr_plot_classifier_comparison_001.png)

1. **Wybierz prawdziwy problem klasyfikacyjny**
2. **Zaimplementuj pipeline dla obu algorytmów**:
   ```python
   # Przykład pipeline
   svm_pipeline = Pipeline([
       ('scaler', StandardScaler()),
       ('svm', SVC())
   ])
   
   xgb_pipeline = Pipeline([
       ('xgb', XGBClassifier())
   ])
   ```
3. **Porównaj**:
   - Dokładność (accuracy, precision, recall, F1)
   - Czas trenowania
   - Interpretację wyników
   - Stabilność wyników (uruchom kilka razy)

---

## Zadanie 4: Analiza błędów 🔍


1. **Dla wybranego modelu przeanalizuj błędy klasyfikacji**:
   - Stwórz confusion matrix
   - Znajdź false positives i false negatives
   - Wizualizuj błędnie sklasyfikowane próbki

2. **Sprawdź, czy istnieje wzór w błędnie klasyfikowanych próbkach**:
   - Czy mają podobne wartości cech?
   - Czy są outlierami?
   - Czy znajdują się blisko granicy decyzyjnej?

3. **Zaproponuj sposoby poprawy modelu**:
   - Feature engineering
   - Zmiana hiperparametrów
   - Usunięcie outlierów
   - Zbieranie więcej danych

---

## Zadanie bonusowe: Ensemble obu algorytmów 🎆


**Spróbuj połączyć SVM i XGBoost** w jedno rozwiązanie:

```python
from sklearn.ensemble import VotingClassifier

# Voting Classifier
voting_clf = VotingClassifier(
    estimators=[('svm', svm_pipeline), ('xgb', xgb_pipeline)],
    voting='soft'  # lub 'hard'
)
```

**Sprawdź, czy ensemble daje lepsze wyniki niż pojedyncze modele!**

---

## 📝 Wskazówki do wszystkich zadań:

1. **Dokumentuj swoje eksperymenty** - zapisuj parametry i wyniki
2. **Wizualizuj wyniki** - użyj matplotlib i seaborn
3. **Interpretuj wyniki** - nie tylko patrzeć na accuracy
4. **Testuj na różnych zbiorach danych** - każdy algoritm ma swoje mocne strony
5. **Mierz czas wykonania** - w praktyce to często kluczowe

# Podsumowanie

![ML Algorithms Comparison](https://scikit-learn.org/stable/_images/sphx_glr_plot_classifier_comparison_001.png)

## SVM - Kluczowe punkty:

![SVM Summary](https://scikit-learn.org/stable/_images/sphx_glr_plot_svm_margin_001.png)

- **Znajduje optymalną granicę** maksymalizując margines
- **Kernele** umożliwiają modelowanie nieliniowych zależności
- **Parametr C** kontroluje trade-off między marginesem a błędami
- **Parametr gamma (RBF)** kontroluje złożoność granicy decyzyjnej
- **Działa dobrze** na wysokowymiarowych danych
- **Deterministyczny** - zawsze te same wyniki dla tych samych danych

## XGBoost - Kluczowe punkty:
![XGBoost Summary ](https://miro.medium.com/v2/resize:fit:1400/1*Sc1OVfgV_jYTIDltlmirIw.png)


- **Buduje model sekwencyjnie**, poprawiając błędy poprzedników
- **Wiele mechanizmów regularyzacji** przeciwdziałających overfitting
- **Automatycznie obsługuje braki danych**
- **Zapewnia ważność cech** pomagającą w interpretacji
- **Skalowalny i efektywny** na dużych zbiorach danych
- **Elastyczny** - wiele opcji tuningu i customizacji

## Wybór algorytmu - Praktyczne wskazówki:

### Użyj **SVM** gdy:
✅ Małe/średnie dane (< 10k próbek)  
✅ Wysokie wymiary (> 1000 cech)  
✅ Potrzebujesz stabilnych, powtarzalnych wyników  
✅ Dane są czyste (bez braków)  
✅ Teoretyczne uzasadnienie jest ważne  

### Użyj **XGBoost** gdy:
✅ Duże dane tabelaryczne (> 10k próbek)  
✅ Potrzebujesz interpretacji (feature importance)  
✅ Dane mają braki lub są zaszumione  
✅ Chcesz szybkie prototypowanie  
✅ Uczestniczysz w konkursach ML  


---

**Pamiętaj**: Nie ma jednego "najlepszego" algorytmu. Skuteczność zależy od konkretnego problemu, danych i kontekstu biznesowego. Najlepsze podejście to przetestowanie kilku algorytmów i wybór tego, który najlepiej spełnia Twoje wymagania pod względem dokładności, interpretowalności i efektywności obliczeniowej.